# Semantic Kernel with Azure OpenAI ⚡

This notebook is a simple demonstration of how to use the [Semantic Kernel](https://learn.microsoft.com/semantic-kernel/overview/?WT.mc_id=AZ-MVP-5003203) with [Azure OpenAI](https://azure.microsoft.com/products/ai-services/openai-service?WT.mc_id=AZ-MVP-5003203). 

## Install requirements
Make sure you have configured the necessary settings for the Azure OpenAI Connector. See the [documentation](https://github.com/microsoft/semantic-kernel/blob/main/python/README.md#openai--azure-openai-api-keys).

We only need to install the Semantic Kernel SDK from pypi.org

In [ ]:
%pip install semantic-kernel

## Initialize the Kernel

In [27]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

kernel = Kernel()
service_id="chat-gpt"

kernel.add_service(
  AzureChatCompletion(
      service_id=service_id,
  )
)

## Add Plugins
The following script will add all plugins from the `plugins_directory` to the kernel:

In [48]:
plugins_directory = "Plugins"
azure_plugin = kernel.add_plugin(parent_directory=plugins_directory, plugin_name="AzurePlugin")

## Run the AzureCli Plugin 

In [ ]:
result = await kernel.invoke(azure_plugin["AzureCliCommand"], input="retrieve all azure open ai api keys")
print(result)

## Run the Azure Advisor Plugin

In [ ]:
project_description="""
    The project is an e-commerce platform that needs to handle high traffic during seasonal sales. 
    It should be able to scale automatically, manage containerized microservices, and provide secure payment processing. 
    The application requires high availability across multiple regions with failover capabilities. 
    Cost optimization is a priority, but the solution should also ensure smooth handling of peak traffic. 
    The team prefers minimal infrastructure management but needs some control over deployment and scaling configurations.
"""

result = await kernel.invoke(azure_plugin["AzureAdvisor"], input=project_description)
print(result)

In [ ]:

project_description="""
    The project is a real-time notification system for a mobile app. 
    It needs to process events triggered by user activity and scale automatically based on demand. 
    The team wants to minimize infrastructure management and prefers a serverless solution with event-driven architecture. 
    The application should handle intermittent high loads but remain cost-efficient when usage is low. 
    Security and seamless integration with other Azure services, like storage and databases, are also important.
"""

result = await kernel.invoke(azure_plugin["AzureAdvisor"], input=project_description)
print(result)

## Add native code as a plugin
The plugin described below allows us to **execute Azure CLI** commands.

In [36]:
import subprocess

from semantic_kernel.functions import kernel_function

class ExecuteAzureCliCommand:
    """
    Description: This plugin is used to execute Azure CLI commands
    """

    @kernel_function(
        description="Execute Azure CLI command",
        name="ExecuteAzureCliCommand",
    )
    def execute_azurecli_command (self, input: str) -> str:
        """
        Execute Azure CLI command
        """
        try:
            command = input.replace("az", "").strip() # remove az from the input
            result = subprocess.run(['az'] + command.split(), capture_output=True, text=True, check=True, shell=True)

            print(result.stdout)
        except subprocess.CalledProcessError as e:
            print(f"Error executing Azure CLI command: {e}")


### Add the native plugin to the kernel

In [37]:
execute_azurecli_plugin = kernel.add_plugin(plugin=ExecuteAzureCliCommand(), plugin_name="ExecuteAzureCliCommand")

In [ ]:
execute_azurecli_fnc = execute_azurecli_plugin["ExecuteAzureCliCommand"]
azurecli_output = await execute_azurecli_fnc(kernel,input="account show")
print(azurecli_output)


## Using the Planner 🪄

In [ ]:
from semantic_kernel.planners.function_calling_stepwise_planner import (
    FunctionCallingStepwisePlanner,
    FunctionCallingStepwisePlannerOptions,
)

question = "List all ressource groups"

options = FunctionCallingStepwisePlannerOptions(
    max_iterations=5,
    max_tokens=4000,
)

planner = FunctionCallingStepwisePlanner(service_id=service_id, options=options)
result = await planner.invoke(kernel, question)

### Print the plan

In [ ]:
print(f"Chat history: {result.chat_history}\n")